In [ ]:
import carla  # CARLA simulator library
import time   # For delays
import math   # For mathematical calculations
import random # For random selection of spawn points
import cv2    # For displaying camera feed (optional for debug visualization)
import numpy as np  # For handling image data
import sys    # To add CARLA API path

# Append the CARLA Python API path (adjust this path to your CARLA installation)
sys.path.append('Z:\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\carla')

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Connect to the CARLA server
client.set_timeout(10.0)  # Set a timeout for the connection

# Access the simulation world
world = client.get_world()

# Constants for controlling the vehicle
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance (in km/h)
MAX_STEER_DEGREES = 80  # Maximum allowed steering angle in degrees
STEERING_CONVERSION = 50  # Conversion factor for steering input (adjust as necessary)
CAMERA_POS_X = -5  # Camera position behind the vehicle (in meters)
CAMERA_POS_Z = 3   # Camera height above the vehicle (in meters)

# Font settings for displaying telemetry data (OpenCV)
font = cv2.FONT_HERSHEY_SIMPLEX
org1 = (30, 30)  # Position for speed display
org2 = (30, 70)  # Position for steering angle display
color = (255, 255, 255)  # White text color
fontScale = 0.5
thickness = 1

# Functions for vehicle and simulation control

def spawn_pedestrians(num_pedestrians=10):
    """
    Spawn pedestrians at random spawn points while avoiding collisions.
    
    :param num_pedestrians: Number of pedestrians to spawn.
    """
    pedestrian_bp = world.get_blueprint_library().filter('walker.pedestrian.*')
    spawn_points = world.get_map().get_spawn_points()

    for _ in range(num_pedestrians):
        transform = random.choice(spawn_points)
        # Attempt to spawn pedestrian
        pedestrian = world.try_spawn_actor(random.choice(pedestrian_bp), transform)
        if pedestrian is None:
            print("Failed to spawn pedestrian, trying another position.")
        else:
            print(f"Pedestrian spawned at {transform.location}")

def spawn_traffic(num_vehicles=10):
    """
    Spawn vehicles at random spawn points while avoiding collisions.
    
    :param num_vehicles: Number of vehicles to spawn.
    """
    traffic_bp = world.get_blueprint_library().filter('vehicle.*')
    spawn_points = world.get_map().get_spawn_points()

    for _ in range(num_vehicles):
        transform = random.choice(spawn_points)
        # Attempt to spawn vehicle
        vehicle = world.try_spawn_actor(random.choice(traffic_bp), transform)
        if vehicle is None:
            print("Failed to spawn vehicle, trying another position.")
        else:
            print(f"Vehicle spawned at {transform.location}")
            vehicle.set_autopilot(True)  # Enable autopilot for spawned traffic vehicles

def maintain_speed(speed_kmh):
    """
    Adjust throttle to maintain the preferred speed.

    :param speed_kmh: Current speed of the vehicle in km/h.
    :return: Throttle value (0 to 1).
    """
    if speed_kmh >= PREFERRED_SPEED:
        return 0  # No throttle if at or above preferred speed
    elif speed_kmh < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9  # Apply more throttle if significantly below preferred speed
    else:
        return 0.4  # Moderate throttle adjustment

def get_angle(vehicle, waypoint):
    """
    Calculate the angle between the vehicle's forward direction and the direction to a waypoint.

    :param vehicle: The vehicle actor.
    :param waypoint: The target waypoint.
    :return: Angle in degrees.
    """
    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_forward = vehicle_transform.get_forward_vector()

    waypoint_location = waypoint.transform.location

    # Calculate the angle between the vehicle and the waypoint
    dx = waypoint_location.x - vehicle_location.x
    dy = waypoint_location.y - vehicle_location.y
    waypoint_angle = math.degrees(math.atan2(dy, dx))
    vehicle_angle = math.degrees(math.atan2(vehicle_forward.y, vehicle_forward.x))

    # Normalize the angle difference
    angle_diff = waypoint_angle - vehicle_angle
    if angle_diff > 180:
        angle_diff -= 360
    elif angle_diff < -180:
        angle_diff += 360

    return angle_diff

def get_next_waypoint(vehicle):
    """
    Get the next waypoint for the vehicle.

    :param vehicle: The vehicle actor.
    :return: The next waypoint.
    """
    vehicle_location = vehicle.get_transform().location
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)

    if waypoint:
        next_waypoints = waypoint.next(2.0)  # Get the next waypoint 2 meters ahead
        if next_waypoints:
            return next_waypoints[0]
    return None

def draw_path(vehicle, min_distance=None):
    """
    Draw the vehicle's path on the map (for visualization).

    :param vehicle: The vehicle actor.
    :param min_distance: Distance to the nearest object, optional for display.
    """
    vehicle_location = vehicle.get_transform().location
    world.debug.draw_point(vehicle_location, size=0.2, color=carla.Color(255, 0, 0), life_time=0.1)

    # Get the current waypoint and draw the path ahead
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    if waypoint:
        # Get waypoints ahead until the end of the lane
        waypoints = waypoint.next_until_lane_end(2.0)
        for wp in waypoints:
            world.debug.draw_point(wp.transform.location, size=0.1, color=carla.Color(0, 255, 0), life_time=1.0)
            if wp.previous(2.0):
                for prev_wp in wp.previous(2.0):
                    world.debug.draw_line(
                        wp.transform.location, prev_wp.transform.location,
                        thickness=0.05, color=carla.Color(0, 0, 255), life_time=1.0
                    )

def camera_callback(image, data_dict):
    """
    Process camera images into a format suitable for display.

    :param image: CARLA image object.
    :param data_dict: Dictionary to store processed image data.
    """
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Main Script
vehicle = None  # Initialize vehicle variable to avoid reference before assignment error
try:
    # Spawn traffic and pedestrians first
    spawn_pedestrians(10)  # Spawn 10 pedestrians
    spawn_traffic(10)  # Spawn 10 vehicles

    # Wait for the user to press 'C' to spawn the vehicle
    print("Press 'C' to spawn your vehicle.")
    while True:
        if cv2.waitKey(1) & 0xFF == ord('c'):  # Wait for user to press 'C'
            print("Spawning vehicle...")

            # Get the blueprint library and spawn a vehicle
            blueprint_library = world.get_blueprint_library()
            vehicle_bp = blueprint_library.filter('mini')[0]  # Select a vehicle blueprint (e.g., Mini)
            spawn_points = world.get_map().get_spawn_points()

            # Try to spawn the vehicle at a random, unoccupied spawn point
            spawn_point = random.choice(spawn_points)
            vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)  # Try to spawn the vehicle

            if vehicle is None:
                print("Failed to spawn vehicle, trying another position.")
            else:
                print(f"Vehicle spawned at {spawn_point.location}")
                break  # Exit the loop once the vehicle is successfully spawned

    # Attach a camera to the vehicle
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')  # Set camera resolution
    camera_bp.set_attribute('image_size_y', '360')
    camera_transform = carla.Transform(carla.Location(x=CAMERA_POS_X, z=CAMERA_POS_Z))
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

    # Listen to the camera feed
    data_dict = {'image': None}  # Initialize dictionary to store image
    camera.listen(lambda image: camera_callback(image, data_dict))

    # Wait for user input to start the car
    print("Press 'S' to start the car.")
    while True:
        if cv2.waitKey(1) & 0xFF == ord('s'):  # Wait for user to press 'S'
            print("Starting the car...")

            # Ask the user if they want manual or automatic control
            print("Press 'M' for Manual Control or 'A' for Automatic Control.")
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == ord('m'):  # Manual control
                    is_auto_control = False
                    print("Manual control selected.")
                    break
                elif key == ord('a'):  # Automatic control
                    is_auto_control = True
                    print("Automatic control selected.")
                    break

            break  # Exit the loop after car has been started

    # Main simulation loop
    while True:
        # Get the current vehicle speed in km/h
        velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)

        # Get the nearest object distance (just for illustration)
        min_distance = 10  # You can implement the actual distance calculation here

        # Control the car based on the selected mode (manual or automatic)
        if is_auto_control:
            throttle = maintain_speed(speed_kmh)  # Automatic throttle control
        else:
            throttle = 0.5  # Manual mode, keep throttle constant for simplicity

        # Get the next waypoint for the vehicle
        next_waypoint = get_next_waypoint(vehicle)
        if next_waypoint:
            # Calculate the angle to the next waypoint
            angle_to_waypoint = get_angle(vehicle, next_waypoint)

            # Limit the steering angle
            if angle_to_waypoint > MAX_STEER_DEGREES:
                steer = MAX_STEER_DEGREES / STEERING_CONVERSION
            elif angle_to_waypoint < -MAX_STEER_DEGREES:
                steer = -MAX_STEER_DEGREES / STEERING_CONVERSION
            else:
                steer = angle_to_waypoint / STEERING_CONVERSION

            # Apply the vehicle control inputs
            control = carla.VehicleControl(throttle=throttle, steer=steer)
            vehicle.apply_control(control)

            # Show camera feed and vehicle data
            if data_dict['image'] is not None:
                img = data_dict['image']
                img = cv2.putText(img, f'Speed: {speed_kmh:.1f} km/h', org1, font, fontScale, color, thickness)
                img = cv2.putText(img, f'Steering Angle: {angle_to_waypoint:.1f}', org2, font, fontScale, color, thickness)
                cv2.imshow("Camera Feed", img)

        # Check for 'q' key press to quit the simulation
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    print("Exiting simulation.")
    cv2.destroyAllWindows()
    if 'camera' in locals():
        camera.stop()  # Stop the camera if it's running
    if 'vehicle' in locals():
        vehicle.destroy()  # Destroy the vehicle when done


Pedestrian spawned at Location(x=32.045444, y=13.273029, z=0.600000)
Pedestrian spawned at Location(x=-41.749878, y=-41.373684, z=0.600000)
Pedestrian spawned at Location(x=-28.581730, y=140.535553, z=0.600000)
Pedestrian spawned at Location(x=44.028950, y=52.548698, z=0.600000)
Pedestrian spawned at Location(x=109.946968, y=-17.187952, z=0.599999)
Pedestrian spawned at Location(x=57.403645, y=28.343533, z=0.600000)
Pedestrian spawned at Location(x=11.176284, y=-64.398766, z=0.600000)
Pedestrian spawned at Location(x=106.513153, y=-21.554596, z=0.900000)
Pedestrian spawned at Location(x=-52.133560, y=-40.180298, z=0.600000)
Pedestrian spawned at Location(x=10.912545, y=-57.401386, z=0.600000)
Vehicle spawned at Location(x=40.389641, y=41.945496, z=0.600000)
Vehicle spawned at Location(x=-68.735168, y=129.303848, z=0.600000)
Vehicle spawned at Location(x=98.800659, y=82.890846, z=0.600000)
Vehicle spawned at Location(x=43.373123, y=16.909227, z=0.600000)
Vehicle spawned at Location(x=-7

In [3]:
import subprocess
import time
import os
import signal

# Path to the CARLA executable
carla_executable_path = "Z:/CARLA_0.9.14/WindowsNoEditor/CarlaUE4.exe"  # Change this to your CARLA executable path

# Function to kill the current CARLA simulation
def kill_carla_sim():
    try:
        # Find the CARLA process by name and kill it
        for proc in os.popen('tasklist').readlines():
            if 'CarlaUE4.exe' in proc:
                pid = int(proc.split()[1])  # Get the PID
                os.kill(pid, signal.SIGTERM)  # Kill the process
                print("CARLA simulation stopped.")
                return True
        print("No running CARLA simulation found.")
        return False
    except Exception as e:
        print(f"Error stopping CARLA simulation: {e}")
        return False

# Function to start the CARLA simulation
def start_carla_sim():
    try:
        print("Starting CARLA simulation...")
        subprocess.Popen(carla_executable_path)  # Start the CARLA server
        time.sleep(5)  # Wait a few seconds for CARLA to fully start
        print("CARLA simulation started.")
    except Exception as e:
        print(f"Error starting CARLA simulation: {e}")

# Example usage
def restart_carla():
    if kill_carla_sim():  # If the CARLA simulation is running, stop it
        time.sleep(2)  # Wait a bit before restarting
    start_carla_sim()  # Start the CARLA simulation

# Restart CARLA simulation
restart_carla()


No running CARLA simulation found.
Starting CARLA simulation...
CARLA simulation started.
